In [ ]:
from torch.utils.data import DataLoader, Dataset
import nltk
from bpemb import BPEmb
import string
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
from torch import nn
import torch

In [19]:
!pip install datasets
!pip install bpemb

dataset = load_dataset("copenlu/answerable_tydiqa")
nltk.download('punkt')

train_set = dataset["train"]
validation_set = dataset["validation"]

def get_answer_start(row):
  return row["annotations"]["answer_start"][0]

def get_answer(row):
  return row["annotations"]["answer_text"][0]

def get_document(row):
  return row["document_plaintext"]

def get_question(row):
  return row["question_text"]

def oracle(answer, document):
  return answer != "" and answer in document

def get_language(dataset, lang):
  return [row for row in dataset if row['language'] == lang]

train_arabic = get_language(train_set, "arabic")
val_arabic = get_language(validation_set, "arabic")

train_bengali = get_language(train_set, "bengali")
val_bengali = get_language(validation_set, "bengali")

train_indonesian = get_language(train_set, "indonesian")
val_indonesian = get_language(validation_set, "indonesian")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
vocab_size = 25000
encoding_dim = 100

bpemb_ar = BPEmb(lang='ar', dim=encoding_dim, vs=vocab_size)
bpemb_bn = BPEmb(lang='bn', dim=encoding_dim, vs=vocab_size)
bpemb_in = BPEmb(lang='id', dim=encoding_dim, vs=vocab_size)

In [20]:
train_arabic_doc = [get_document(row) for row in train_arabic]
train_arabic_question = [get_question(row) for row in train_arabic]

train_bengali_doc = [get_document(row) for row in train_bengali]
train_bengali_question = [get_question(row) for row in train_bengali]

train_indonesian_doc = [get_document(row) for row in train_indonesian]
train_indonesian_question = [get_question(row) for row in train_indonesian]

val_arabic_doc = [get_document(row) for row in val_arabic]
val_arabic_question = [get_question(row) for row in val_arabic]

val_bengali_doc = [get_document(row) for row in val_bengali]
val_bengali_question = [get_question(row) for row in val_bengali]

val_indonesian_doc = [get_document(row) for row in val_indonesian]
val_indonesian_question = [get_question(row) for row in val_indonesian]

In [ ]:
def text_to_ids(text, tokenizer):
    input_ids = tokenizer.encode_ids_with_eos(text)
    return input_ids, len(input_ids)

def pad_input(input):
    input_ids = [i[0] for i in input]
    seq_lens = [i[1] for i in input]

    max_length = max(seq_lens)

    input_ids = [(i + [25000] * (max_length - len(i))) for i in input_ids]

    # Make sure each sample is max_length long
    assert (all(len(i) == max_length for i in input_ids))
    return torch.tensor(input_ids), torch.tensor(seq_lens)

In [ ]:
class DatasetReader(Dataset):
  def __init__(self, data, tokenizer):
    self.data = data
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    row = self.data[idx]
    input_ids, seq_lens = text_to_ids(row, self.tokenizer)
    return input_ids, seq_lens


In [ ]:
class LSTMNetwork(nn.Module):
    def __init__(
            self,
            pretrained_embeddings,
            vocab_size: int,
            num_layers,
            hidden_dim: int,
            dropout_rate: float = 0.1,
            bidirectional: bool = False
    ):
        super(LSTMNetwork, self).__init__()

        self.vocab_size = vocab_size

        self.embeddings = nn.Embedding.from_pretrained(pretrained_embeddings, padding_idx=pretrained_embeddings.shape[0] - 1)

        self.lstm = nn.LSTM(
                pretrained_embeddings.shape[1],
                hidden_dim,
                num_layers,
                batch_first=True,
                dropout=dropout_rate,
                bidirectional=bidirectional)

        self.dropout = nn.Dropout(dropout_rate)

        self.output_layer = nn.Linear(2*hidden_dim if bidirectional else hidden_dim, vocab_size)

        # Initialize the weights of the model
        self._init_weights()

    def _init_weights(self):
        all_params = list(self.lstm.named_parameters()) + list(self.output_layer.named_parameters())
        for n,p in all_params:
            if 'weight' in n:
                nn.init.xavier_normal_(p)
            elif 'bias' in n:
                nn.init.zeros_(p)

    def forward(self, inputs, input_lens):

        embeds = self.embeddings(inputs)

        # Pack padded: This is necessary for padded batches input to an RNN
        lstm_in = nn.utils.rnn.pack_padded_sequence(
            embeds,
            input_lens.cpu(),
            batch_first=True,
            enforce_sorted=False
        )

        lstm_out, _ = self.lstm(lstm_in)

        lstm_out, _ = nn.utils.rnn.pad_packed_sequence(lstm_out, batch_first=True)

        lin_in = self.dropout(lstm_out)

        targets = torch.flatten(inputs.clone())

        output = self.output_layer(lin_in)
        logits = output.view(-1, self.vocab_size)

        loss_fn = nn.CrossEntropyLoss()
        loss = loss_fn(logits, targets)

        return (output, loss)


In [ ]:
from math import exp

def train(
    model,
    dl,
    optimizer: torch.optim.Optimizer,
    n_epochs: int,
    device,
    patience: int = 10
):
  # Keep track of the loss and best accuracy
  best_ppl = float("inf")
  pcounter = 0

  # Iterate through epochs
  for ep in range(n_epochs):

    losses_epoch = []

    for batch in tqdm(dl):
      model.train()
      optimizer.zero_grad()

      batch = tuple(t.to(device) for t in batch)
      inputs = batch[0]
      seq_lens = batch[1]

      (output, loss) = model(inputs, seq_lens)

      losses_epoch.append(loss.item())

      loss.backward()

      #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()

    avg_loss = sum(losses_epoch) / len(losses_epoch)
    ppl = exp(avg_loss)

    print(f'Avg Loss / Perplexity at epoch {ep}: {avg_loss}/{ppl}')

    # Keep track of the best model based on the accuracy
    if ppl < best_ppl:
      torch.save(model.state_dict(), 'best_model')
      best_ppl = ppl
      pcounter = 0
    else:
      pcounter += 1
      if pcounter == patience:
        break

  model.load_state_dict(torch.load('best_model'))
  return model, best_ppl

In [16]:
from torch.optim import Adam

num_layers = 2
hidden_dim = 100
dropout_rate = 0.1
lr = 0.01
n_epochs = 100
#batch_size = 8

bidirectional = True

patience = 5

runs = [#("Arabic document", train_arabic_doc, bpemb_ar, 4),
        #("Arabic question", train_arabic_question, bpemb_ar, 32),
        #("Bengali document", train_bengali_doc, bpemb_bn, 4),
        ("Bengali question", train_bengali_question, bpemb_bn, 32),
        #("Indonesian document", train_indonesian_doc, bpemb_in, 4),
        ("Indonesian question", train_indonesian_question, bpemb_in, 32),
]

device = torch.device("cpu")
if torch.cuda.is_available():
  device = torch.device("cuda")

results = []

for (tag, ds, embs, batch_size) in runs:

  pretrained_embeddings = torch.Tensor(np.concatenate([embs.emb.vectors, np.zeros(shape=(1,100))], axis=0))
  vocabulary = embs.emb.index_to_key + ['[PAD]']

  model = LSTMNetwork(pretrained_embeddings, len(vocabulary), num_layers, hidden_dim, dropout_rate, bidirectional).to(device)

  reader = DatasetReader(ds, embs)

  dl = DataLoader(reader, batch_size=batch_size, collate_fn=pad_input, shuffle=True, num_workers=2)

  optimizer = Adam(model.parameters(), lr=lr)

  best_model, ppl = train(model, dl, optimizer, n_epochs, device, patience)

  results.append((tag, ppl, best_model))

100%|██████████| 150/150 [00:02<00:00, 58.53it/s]


Avg Loss / Perplexity at epoch 0: 6.747271849314372/851.7319369361895


100%|██████████| 150/150 [00:02<00:00, 51.29it/s]


Avg Loss / Perplexity at epoch 1: 4.029033211072286/56.206545105597435


100%|██████████| 150/150 [00:02<00:00, 66.31it/s]


...

Avg Loss / Perplexity at epoch 26: 0.008926175974706626/1.0089661330830453


100%|██████████| 357/357 [00:05<00:00, 62.16it/s]


Avg Loss / Perplexity at epoch 27: 0.007686338283568986/1.0077159540118816


100%|██████████| 357/357 [00:04<00:00, 83.48it/s]

Avg Loss / Perplexity at epoch 28: 0.00898644798425488/1.0090269473321325


In [24]:

reader = DatasetReader(val_bengali_question, bpemb_bn)
dl = DataLoader(reader, batch_size=batch_size, collate_fn=pad_input, shuffle=True, num_workers=2)

_, t_ppl, model = results[0]

losses = []
for batch in tqdm(dl):
  model.eval()
  optimizer.zero_grad()
  batch = tuple(t.to(device) for t in batch)
  inputs = batch[0]
  seq_lens = batch[1]
  (output, loss) = model(inputs, seq_lens)
  losses.append(loss.item())

avg_loss = sum(losses) / len(losses)
ppl = exp(avg_loss)
print()
print(ppl)

100%|██████████| 7/7 [00:00<00:00, 24.29it/s]


4.035701842961149


In [26]:
reader = DatasetReader(val_indonesian_question, bpemb_in)
dl = DataLoader(reader, batch_size=batch_size, collate_fn=pad_input, shuffle=True, num_workers=2)

_, t_ppl, model = results[1]

losses = []
for batch in tqdm(dl):
  model.eval()
  optimizer.zero_grad()
  batch = tuple(t.to(device) for t in batch)
  inputs = batch[0]
  seq_lens = batch[1]
  (output, loss) = model(inputs, seq_lens)
  losses.append(loss.item())

avg_loss = sum(losses) / len(losses)
ppl = exp(avg_loss)
print()
print(ppl)

100%|██████████| 38/38 [00:01<00:00, 32.42it/s]


4.51219946742593
